In [ ]:
!pip install requests gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 393.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/

In [ ]:
import requests
import pandas as pd
import gradio as gr

# API key and user ID for the new API
api_key = 'KGSjxB1uptfSk8I8A7ciCuNT9Xa3qWC3'
external_user_id = 'plugin-1717464304'

In [ ]:
# Step 1: Create a chat session
def create_chat_session():
    create_session_url = 'https://api.on-demand.io/chat/v1/sessions'
    create_session_headers = {
        'apikey': api_key
    }
    create_session_body = {
        "pluginIds": [],
        "externalUserId": external_user_id
    }
    # Make the request to create a chat session
    response = requests.post(create_session_url, headers=create_session_headers, json=create_session_body)
    response_data = response.json()
    session_id = response_data['data']['id']
    return session_id

In [ ]:
# Step 2: Submit a query to the API
def submit_query(session_id, query):
    submit_query_url = f'https://api.on-demand.io/chat/v1/sessions/{session_id}/query'
    submit_query_headers = {
        'apikey': api_key
    }
    submit_query_body = {
        "endpointId": "predefined-openai-gpt4o",
        "query": query,
        "pluginIds": ["plugin-1712327325", "plugin-1713962163"],
        "responseMode": "sync"
    }
    response = requests.post(submit_query_url, headers=submit_query_headers, json=submit_query_body)
    return response.json()

In [ ]:
# Step 3: Create a prompt with context examples and explicitly ask for diagnosis, treatment, etc.
def create_prompt(patient_info, query_type, context_examples):
    """Creates a well-engineered prompt to ask for diagnosis, most probable diagnosis, treatment, lifestyle modifications, and medications."""
    prompt_examples = {
        "diagnosis": """Example: A 45-year-old female presents with severe daily headaches, occasional dizziness, and hypertension. MRI shows mild optic nerve swelling.
        Symptoms: Headaches, dizziness, hypertension, optic nerve swelling
        Possible diagnoses: Idiopathic intracranial hypertension (IIH), migraine, pseudotumor cerebri
        Most probable diagnosis: IIH due to elevated intracranial pressure
        Treatment plan: Acetazolamide to reduce ICP, regular eye exams to monitor for vision changes, weight loss if obesity is a factor.
        Lifestyle modifications: Weight loss, low-sodium diet, regular check-ups
        Medications: Acetazolamide, aspirin, diuretics if needed
        """,

        "treatment": """Example: A 55-year-old female presents with shortness of breath, chest tightness, and occasional vision disturbances. MRI shows small areas of white matter hyperintensities.
        Symptoms: Shortness of breath, chest pain, white matter hyperintensities
        Possible diagnoses: Transient ischemic attack (TIA), small vessel ischemic disease, coronary artery disease (CAD)
        Most probable diagnosis: TIA due to reduced blood flow in the brain
        Treatment plan: Aspirin or other anticoagulants, statins for CAD, lifestyle changes to manage hypertension.
        Lifestyle modifications: Healthy diet, regular exercise, smoking cessation
        Medications: Aspirin, statins, beta-blockers
        """
    }

    example = prompt_examples.get(query_type, "")
    prompt = f"""
    Patient Information: {patient_info}
    Please provide the following:
    - Diagnosis
    - Most probable diagnosis
    - Treatment plan
    - Lifestyle modifications
    - Medications
    """

    return prompt

In [ ]:
# Step 4: Log patient data and responses
def log_patient_data(patient_info, query_type, response_text):
    """Logs patient query and generated response."""
    log_entry = {
        "patient_info": patient_info,
        "query_type": query_type,
        "response": response_text
    }
    return log_entry

In [ ]:
# Step 5: Format the response for better readability and ensure each section is highlighted
def format_response(response_text):
    """Formats the response to improve readability and ensure the diagnosis, treatment plan, etc. are properly displayed."""
    # Replace "\n\n" with clear breaks and ensure sections are properly formatted
    formatted_response = response_text.replace("\\n\\n", "\n\n").replace("\\n", "\n")

    # Add indentation or section titles for the expected fields
    sections = ["Diagnosis", "Most probable diagnosis", "Treatment plan", "Lifestyle modifications", "Medications"]
    for section in sections:
        formatted_response = formatted_response.replace(section, f"\n--- {section} ---\n")

    # Optionally, you could enhance readability further with line breaks
    formatted_response = "\n--- Patient Recommendations ---\n" + formatted_response.strip()

    print(formatted_response)

In [ ]:
def submit_query(session_id, query):
    """Submits a query to the API and returns the response text."""
    submit_query_url = f'https://api.on-demand.io/chat/v1/sessions/{session_id}/query'
    submit_query_headers = {
        'apikey': 'KGSjxB1uptfSk8I8A7ciCuNT9Xa3qWC3'
    }
    submit_query_body = {
        "endpointId": "predefined-openai-gpt4o",
        "query": query,
        "pluginIds": ["plugin-1712327325", "plugin-1713962163"],
        "responseMode": "sync"
    }

    response = requests.post(submit_query_url, headers=submit_query_headers, json=submit_query_body)

    # Debugging - print the entire response
    print(f"Full Response: {response.json()}")

    return response.json()

In [ ]:
# Step 6: Generate and submit a query with context examples, log responses
def generate_query_response(patient_info, query_type, logs, context_examples):
    session_id = create_chat_session()  # Start the session
    prompt = create_prompt(patient_info, query_type, context_examples)
    print(f"Generated Prompt: {prompt}")  # Debugging line
    response = submit_query(session_id, prompt)  # Submit the query to the new API

    if 'data' in response and 'answer' in response['data']:
        answer = response['data']['answer']  # Extract only the 'answer' field
        format_response(answer)  # Format and print the answer text
        logs.append(log_patient_data(patient_info, query_type, answer))  # Log the answer
        return answer
    else:
        print("No valid response received. Please try again.")

In [ ]:
# Context examples (same as before)
context_examples = [
    {
        "input": "A 45-year-old female presents with severe daily headaches, occasional dizziness, and hypertension. MRI shows mild optic nerve swelling.",
        "output": """
        Symptoms: Headaches, dizziness, hypertension, optic nerve swelling
        Possible diagnoses: Idiopathic intracranial hypertension (IIH), migraine, pseudotumor cerebri
        Most probable diagnosis: Idiopathic intracranial hypertension (IIH)
        Management: Acetazolamide to reduce ICP, regular eye exams to monitor for vision changes, weight loss if obesity is a factor
        Complications: Permanent vision loss due to prolonged papilledema
        """
    },
    {
        "input": "A 55-year-old female presents with shortness of breath, chest tightness, and occasional vision disturbances. MRI shows small areas of white matter hyperintensities.",
        "output": """
        Symptoms: Shortness of breath, chest pain, white matter hyperintensities
        Possible diagnoses: Transient ischemic attack (TIA), small vessel ischemic disease, coronary artery disease (CAD)
        Most probable diagnosis: Transient ischemic attack (TIA)
        Treatment: Aspirin or other anticoagulants, statins for CAD, lifestyle changes to manage hypertension
        """
    }
]


In [ ]:
# Enter your prompts here
prompts = [
    {
        "patient_info": "A 48-year-old male presents with intermittent blurred vision, chronic daily headaches, and nausea, worsening over the past 2 months. MRI shows 66% chance of Glioma Tumor and a 20% chance of ischemic stroke",
        "query_type": "diagnosis"
    },
    {
        "patient_info": "A 60-year-old male patient has been experiencing intermittent chest pain, shortness of breath, and unexplained dizziness for the past 3 months. MRI results show normal brain conditions",
        "query_type": "treatment"
    }
]

In [ ]:
# Step 7: Execute and get responses for a patient case
logs = []
patient_info = "A 60-year-old male patient has been experiencing intermittent chest pain, shortness of breath, and unexplained dizziness for the past 3 months. MRI results show normal brain conditions"
query_type = "diagnosis"
output = generate_query_response(patient_info, query_type, logs, context_examples)

Generated Prompt: 
    Patient Information: A 60-year-old male patient has been experiencing intermittent chest pain, shortness of breath, and unexplained dizziness for the past 3 months. MRI results show normal brain conditions
    Please provide the following:
    - Diagnosis
    - Most probable diagnosis
    - Treatment plan
    - Lifestyle modifications
    - Medications
    
Full Response: {'message': 'Chat query submitted successfully', 'data': {'sessionId': '671c262b0b25466bf27c480b', 'messageId': '671c262b0b25466bf27c480c', 'answer': "Based on the information provided:\n\n- **Diagnosis**: The symptoms of intermittent chest pain, shortness of breath, and dizziness in a 60-year-old male could suggest a cardiovascular issue, such as angina or coronary artery disease.\n\n- **Most Probable Diagnosis**: Angina pectoris is the most probable diagnosis, given the symptoms and the patient's age.\n\n- **Treatment Plan**: The treatment plan may involve managing the underlying cardiovascula

In [ ]:
# Step 8: Display log entries
for entry in logs:
    print(entry)

{'patient_info': 'A 60-year-old male patient has been experiencing intermittent chest pain, shortness of breath, and unexplained dizziness for the past 3 months. MRI results show normal brain conditions', 'query_type': 'diagnosis', 'response': "Based on the information provided:\n\n- **Diagnosis**: The symptoms of intermittent chest pain, shortness of breath, and dizziness in a 60-year-old male could suggest a cardiovascular issue, such as angina or coronary artery disease.\n\n- **Most Probable Diagnosis**: Angina pectoris is the most probable diagnosis, given the symptoms and the patient's age.\n\n- **Treatment Plan**: The treatment plan may involve managing the underlying cardiovascular condition, which could include lifestyle changes, medications, and possibly procedures like angioplasty or coronary artery bypass surgery if necessary.\n\n- **Lifestyle Modifications**: \n  - Adopt a heart-healthy diet low in saturated fats, cholesterol, and sodium.\n  - Engage in regular physical act

In [ ]:
filename = "output.txt"

# Open the file in write mode
with open(filename, 'w') as file:
    # Write the text to the file
    file.write(output)

print(f"Text has been written to {filename}.")

Text has been written to output.txt.


In [ ]:
df = pd.DataFrame (logs)
df

,patient_info,query_type,response
0,A 60-year-old male patient has been experienci...,diagnosis,Based on the information provided:\n\n- **Diag...


In [ ]:
import gradio as gr

# Step 7: Create a Gradio interface
def gradio_interface(patient_info, query_type):
    # Initialize logs outside of the function for persistence
    global logs
    logs = []  # This is now persistent across calls
    response = generate_query_response(patient_info, query_type, logs, context_examples)

    if response is not None:
        return response
    else:
        return "An error occurred while generating the response."

# Create Gradio app with enhanced layout
iface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(
            label="Patient Information",
            placeholder="Enter patient details here...",
            lines=5,  # Adjust height for better visibility
            max_lines=10
        ),
        gr.Textbox(
            label="Query Type"
        )
    ],
    outputs=gr.Textbox(label="Response", placeholder="The response will appear here..."),
    title="Medical Query Interface",
    description="Enter patient information and select query type to get diagnosis and treatment recommendations.",
    css="""
        .input-textbox {
            border-radius: 8px;
            border: 2px solid #007bff;
            font-size: 16px;
        }
        .output-textbox {
            border-radius: 8px;
            border: 2px solid #007bff;
            background-color: #f9f9f9;
            font-size: 16px;
        }
        .gr-button {
            background-color: #007bff;
            color: white;
            border-radius: 8px;
        }
        .gr-button:hover {
            background-color: #0056b3;
        }
    """
)


iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://21372a0bf245a6cd3e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
